## Chapter 12
# The Eigenvector

## 12.12 Lab: Pagerank

**Task 12.12.1:** Write a procedure `find_num_links` with the following spec:
* _input:_ A square matrix `L` representing a link structure.
* _output:_ A vector `num_links` whose label set is the column-label set of `L`, such that, for each column-label `c`, entry `c` of `num_links` is the number of nonzero entries in column `c` of `L`.

In [1]:
import sys
sys.path.append('../')

from vec import Vec

def find_num_links(L):
    ones = Vec(L.D[0], {k: 1 for k in L.D[0]})
    return ones * L

**Task 12.12.2:** Write a procedure `make_Markov` with the following spec:
* _input:_ A square matrix `L` representing a link structure as described above.
* _ouptut:_ This procedure does not produce new output, but instead _mutates_ L so that it plays the role of $A_1$.

In [2]:
def make_Markov(L):
    num_links = find_num_links(L)
    for k in L.f.keys():
        if k[0] == k[1] and num_links[k[1]] == 0:
            L[k] = 1
        else:
            L[k] /= num_links[k[1]]
    return L

In [3]:
from pagerank_test import small_links

print(small_links)


       1 2 3 4 5 6
     -------------
 1  |  1 0 0 1 0 0
 2  |  0 0 1 1 1 1
 3  |  0 1 0 0 0 0
 4  |  0 0 0 0 1 0
 5  |  0 0 0 0 0 1
 6  |  0 0 0 0 1 0



In [4]:
print(find_num_links(small_links))


 1 2 3 4 5 6
------------
 1 1 1 2 3 2


In [5]:
print(make_Markov(small_links))


       1 2 3   4     5   6
     ---------------------
 1  |  1 0 0 0.5     0   0
 2  |  0 0 1 0.5 0.333 0.5
 3  |  0 1 0   0     0   0
 4  |  0 0 0   0 0.333   0
 5  |  0 0 0   0     0 0.5
 6  |  0 0 0   0 0.333   0



**Task 12.12.3:** Write a procedure `power_method` with the following spec:
* _input:_
  - the matrix $A_1$, and
  - the desired number of iterations of the power method.
* _output:_ an approximation to the stationary distribution, or at least a scalar multiple of the stationary distribution

Your initial vector can be pretty much anything nonzero. We recommend using an all-ones vector.

In order to see how well the method converges, at each iteration print the ratio

  (norm of $v$ before the iteration)/(norm of $v$ after the iteration)

As the approximation for the eigenvector with eigenvalue 1 gets better , this ratio should get closer to 1.

Test your code using the matrix $A_1$ you obtained for the Thimble-Wide Web. The module `pagerank_test` defines `A2` to allow you to explicitly test whether the vector you get is an approximate eigenvector of $A$.

You should obtain as an eigenvector a scalar multiple of the following vector:
`{1: 0.5222, 2: 0.6182, 3: 0.5738, 4: 0.0705, 5: 0.783, 6: 0.0705}`

In [6]:
def power_method(A1, num_iterations):
    x = Vec(A1.D[0], {k: 1 for k in A1.D[0]})
    for i in range(num_iterations):
        x = A1 * x
        norm = (x * x) ** 0.5
        x = x / norm
    return x

In [7]:
from pagerank_test import A2
print(power_method(small_links * 0.85 + A2 * 0.15, 100))


     1     2     3      4      5      6
---------------------------------------
 0.522 0.618 0.574 0.0705 0.0783 0.0705


**Task 12.12.5:** Write a procedure `wikigoogle` with the following spec:
* _input:_ 
  - A single word `w`.
  - The number `k` of desired results.
  - The pagerank eigenvector `p`
* _output:_ a list of the names of the `k` highest-pagerank wikipedia articles containing that word.

In [8]:
from pagerank import read_data, find_word

links = read_data()

Reading word meta-index
Reading titles
Reading link structure
..................................................................................
Done


In [9]:
def wikigoogle(w, k, p):
    related = find_word(w)
    related.sort(key=lambda x: p[x], reverse=True)
    return related[:k]

**Task 12.12.6:** Use `power_method` to compute the pagerank eigenvector for the wikipedia corpus and try some queries to see the titles of the top few pages: "jordan", "obama", "tiger" and of course "matrix". What do you get for your top few articles? Can you explain why? Are the top ranked results more relevant or important in some sense than, say, the first few articles returned by `find_word` without ranking?

In [10]:
from mat import Mat

A1 = make_Markov(links)
A2 = Mat(A1.D, {k:1/len(A1.D[0]) for k in A1.f.keys()})
A = 0.85*A1 + 0.15*A2

p = power_method(A, 6)

In [11]:
jordan_results = wikigoogle('jordan', 5, p)
obama_results  = wikigoogle('obama', 5, p)
tiger_results  = wikigoogle('tiger', 5, p)
matrix_results = wikigoogle('matrix', 5, p)

In [13]:
jordan_results

['2007', '2006', '2005', 'paris', 'israel']

In [14]:
obama_results

['united states',
 'president of the united states',
 'chicago',
 'democratic party (united states)',
 'illinois']

In [15]:
tiger_results

['france', 'england', '2006', 'india', 'sweden']

In [16]:
matrix_results

['1999', 'internet', 'religion', '1965', 'sydney']

In [17]:
find_word('jordan')[:5]

['alabama', 'altruism', 'asphalt', 'arabic language', 'attila the hun']

In [18]:
find_word('obama')[:5]

['august 27', 'austin, texas', 'african american', 'august 4', 'the bronx']

The results are indeed much better than the simple `find_word` query.  This makes sense since the eigenvector-weighted results are informed by the stationary distribution of a model involving actual links between pages rather than simply alphabetized documents with occurrences of the word.

**Task 12.12.7:** Write a version of the power method that finds an approximation to the stationary distribution of a Markov chain that is biased. The procedure should be called `power_method_biased`. It resembles `power_method` but takes an additional parameter, the label $r$ of the state (i.e. article) to jump to. It should output an approximate eigenvector of the matrix $.55A_1 + .15A_2 + .3A_r$. Try to write it so that $A_r$ is not explicitly created. Remember to test your procedure on the Thimble-Wide Web before trying it on the big dataset.

Compute the stationary distribution of the Markov chain that is biased towards `sport`.

In [22]:
def power_method_biased(A1, A2, num_iterations, r):
    A_biased = 0.55 * A1 + 0.15 * A2 + 0.3 * Mat(A1.D, {k: 1 for k in A1.f.keys() if k[0] == r})
    return power_method(A_biased, num_iterations)

In [23]:
p_biased_sports = power_method_biased(A1, A2, 6, 'sport')

In [24]:
wikigoogle('jordan', 5, p_biased_sports)

['2007', '2006', '2005', 'paris', 'basketball']

In [25]:
wikigoogle('tiger', 5, p_biased_sports)

['france', 'england', '2006', 'india', 'ireland']

## 12.13 Review questions

**What must be true of a matrix $A$ in order for $A$ to have an eigenvalue?**

In order for a matrix $A$ to have an eigenvalue, it must be a square matrix such that there exists a scalar $\lambda$ and a nonzero vector $\boldsymbol{v}$ such that $A\boldsymbol{v} = \lambda\boldsymbol{v}$.

In particular, one special case is that any square matrix over $\mathbb{C}$ has an eigenvalue.

**What are an eigenvalue and eigenvector of a matrix?**

If $\lambda$ is a scalar and $\boldsymbol{v}$ is a nonzero vector such that $A\boldsymbol{v} = \lambda\boldsymbol{v}$, then $\lambda$ is an eigenvalue of $A$ and $\boldsymbol{v}$ is a corresponding eigenvector.

**For what kind of problems are eigenvalues and eigenvectors useful?**

Eigenvalues and eigenvectors are useful for problems that require many matrix multiplications by the same matrix (matrix exponentiation).  In the real world these types of problems often arise in state spaces that evolve over time according to consistent rules. (Discrete dynamic systems.)

**What is a diagonalizable matrix?**

For a matrix to be diagonalizable it must be similar to a diagonal matrix, meaning there is an invertible matrix $S$ such that $S^{-1}AS = \Lambda$ where $\Lambda$ is a diagonal matrix).

**What is an example of a matrix that has eigenvalues but is not diagonalizable?**

$\begin{bmatrix}1 &1 &0\\0 &1 &1\\0 &0 &4\end{bmatrix}$

**Under what conditions is a matrix guaranteed to be diagonalizable?**

If an $n \times n$ matrix has $n$ distinct eigenvalues, it is guaranteed to be diagonalizable.  Also, a symmetric matrix over $\mathbb{R}$, an an upper-triangular matrix are both guaranteed to be diagonalizable.

**What are some advantages of diagonalizable matrices?**

Diagonalizable matrices are easy to exponentiate since they can be decomposed into $S, S^{-1} and \Lambda$ where $\Lambda$ is a diagonal matrix, and thus only requires squaring the diagonal components to square the matrix.  This property also makes change of basis fast.

**Under what conditions does a matrix have linearly independent eigenvectors?**

A matrix has linearly independent eigenvectors if it is diagonalizable. Also, eigenvectors corresponding to _distinct_ eigenvalues are linearly independent.

**What are the advantages to a matrix having linearly independent eigenvectors?**

Having a set of linearly independent eigenvectors that can span the column space of a matrix allows for a useful change of basis that allows for separation of linear transformations such that transformations by an eigenvector can be modeled simply by scaling by the corresponding eigenvalue.

**Under what conditions does a matrix have orthonormal eigenvectors?**

If a matrix is symmetric it will have orthonormal eigenvectors.

**What is the power method? What is it good for?**

The power method is a method for finding an approximation to the largest eigenvector (and thus the largest eigenvalue) of a matrix. It is used when finding an exact eigenvector would be very costly or impossible.

**What is the determinant?**

The determinant is the _signed_ area of the parallelogram (or more generally the signed volume of the parallelepiped for $n$ dimensions) created by $n$ vectors.

**How does the determinant relate to volumes?**

The determinant is the _signed_ volume of a set of vectors, which negates certain dimensions based on vector orientation.  Otherwise it is identical to the volume!

**Which matrices have determinants?**

All square matrices have determinants, but they may be zero or negative.

**Which matrices have nonzero determinants?**

A matrix has a nonzero determinant if it is invertible.

**What do determinants have to do with eigenvalues?**

The numbers that make the characteristic polynomial of a square matrix equal to 0 are the iegenvalues of $A$, where the characteristic polynomial can be found with the determinant.

**What is a Markov chain?**

A Markov chain is a state transition graph where the nodes are states and the directed edges represent transitions between states.  The transitions can be weighted based on a probability distribution where each node's outgoing edge weights are all >= 0 and sum to 1 (to form a valid distribution).

A key characteristic of Markov chains is that their transition distribution at any node is not dependent on previous state - only on the current state and the transition distribution.

**What do Markov chains have to do with eigenvectors?**

The transition function of a markov chain can be represented as a matrix where each entry $M[i, j]$ holds the transition probability from row $i$ to column $j$ (or vice versa depending on convention).

With this representation, the normalized value of any of the largest eigenvector (or any scalar multiple) will give the stationary distribution of the Markov chain (the point of stability in the probability of "landing" on each node).  This will only be true, however, if the transition probability between _every_ pair of nodes is greater than zero.

## 12.14 Problems

**Problem 12.14.1:** For each matrix, find its eigenvalues and associated eigenvectors. Just use cleverness here; no algorithm should be needed.

a) $\begin{bmatrix}1 &2\\1 &0\end{bmatrix}$

$\boldsymbol{v}_1 = [2,1], \lambda_1= 2$  
$\boldsymbol{v}_2 = [-1,1], \lambda_2= -1$

b) $\begin{bmatrix}1 &1\\3 &3\end{bmatrix}$

$\boldsymbol{v}_1 = [1,3], \lambda_1 = 4$  
$\boldsymbol{v}_2 = [-1,1], \lambda_2 = 0$

c) $\begin{bmatrix}6 &0\\0 &6\end{bmatrix}$

$\boldsymbol{v}_1 = [1,0], \lambda_1 = 6$  
$\boldsymbol{v}_2 = [0,1], \lambda_2 = 6$

d) $\begin{bmatrix}0 &4\\4 &0\end{bmatrix}$

$\boldsymbol{v}_1 = [1,1], \lambda_1 = 4$  
$\boldsymbol{v}_2 = [-1,1], \lambda_2= -1$

**Problem 12.14.2:** In each of the following subproblems, we give you a matrix and some of its eigenvalues. Find a corresponding eigenvector.

a) $\begin{bmatrix}7 &-4\\2 &1\end{bmatrix}$ and eigenvalues $\lambda_1 = 5, \lambda_2 = 3$

For $\lambda_1 = 5, \boldsymbol{v}_1 = [2,1]$  
For $\lambda_2 = 3, \boldsymbol{v}_2 = [1,1]$

b) $\begin{bmatrix}4 &0 &0\\2 &0 &3\\0 &1 &2\end{bmatrix}$ and eigenvalues $\lambda_1 = 3, \lambda_2 = -1$

For $\lambda_1 = 3, \boldsymbol{v}_1 = [0,1,1]$  
For $\lambda_2 = -1, \boldsymbol{v}_2 = [0,3,-1]$

**Problem 12.14.3:** Given a matrix and its eigenvectors, find the corresponding eigenvalues:

a) $\begin{bmatrix}1 &2\\4 &3\end{bmatrix}$ and $\boldsymbol{v}_1 = [\frac{1}{\sqrt{2}}, -\frac{1}{\sqrt{2}}]$ and $\boldsymbol{v}_2 = [1, 2]$

For $\boldsymbol{v}_1 = [\frac{1}{\sqrt{2}}, -\frac{1}{\sqrt{2}}]$, $\lambda_1 = -1$  
For $\boldsymbol{v}_2 = [1, 2]$, $\lambda_2 = 5$

b) $\begin{bmatrix}5 &0\\1 &2\end{bmatrix}$ and $\boldsymbol{v}_1 = [0, 1]$ and $\boldsymbol{v}_2 = [3, 1]$

For $\boldsymbol{v}_1 = [0, 1]$, $\lambda_1 = 2$  
For $\boldsymbol{v}_2 = [3, 1]$, $\lambda_1 = 5$

**Problem 12.14.4:** Let $A = \begin{bmatrix}0 &-1\\1 &0\end{bmatrix}$. Two (unnormalized) eigenvectors are $\boldsymbol{v}_1 = \begin{bmatrix}1\\\boldsymbol{i}\end{bmatrix}$ and $\boldsymbol{v}_2 = \begin{bmatrix}1\\-\boldsymbol{i}\end{bmatrix}$.

1. Find the eigenvalue $\lambda_1$ corresponding to eigenvector $\boldsymbol{v}_1$, and show using matrix-vector multiplication that it is indeed the corresponding eigenvalue.

  $\begin{bmatrix}0 &-1\\1 &0\end{bmatrix}\begin{bmatrix}1\\\boldsymbol{i}\end{bmatrix} = \begin{bmatrix}-\boldsymbol{i}\\1\end{bmatrix}$

  $\begin{bmatrix}-\boldsymbol{i}\\1\end{bmatrix} = \lambda_1\begin{bmatrix}1\\\boldsymbol{i}\end{bmatrix}$
  
  $\lambda_1 = \boldsymbol{i}^3$
2. Find the eigenvalue $\lambda_2$ corresponding to eigenvector $\boldsymbol{v}_2$, and show using matrix-vector multiplication that it is indeed the corresponding eigenvalue.

  $\begin{bmatrix}0 &-1\\1 &0\end{bmatrix}\begin{bmatrix}1\\-\boldsymbol{i}\end{bmatrix} = \begin{bmatrix}\boldsymbol{i}\\1\end{bmatrix}$

  $\begin{bmatrix}\boldsymbol{i}\\1\end{bmatrix} = \lambda_2\begin{bmatrix}1\\-\boldsymbol{i}\end{bmatrix}$
  
  $\lambda_2 = \boldsymbol{i}$

**Problem 12.14.5:** Given a matrix $A$

$\begin{bmatrix}1 &2 &5 &7\\2 &9 &3 &7\\1 &0 &2 &2\\7 &3 &9 &1\end{bmatrix}$

a) Use the `power` method to approximate the eigenvector that corresonds to the eigenvalue $\lambda_1$ of largest absolute value.

b) Find an approximation to $\lambda_1$.

c) Using the `eig` procedure in `numpy`, find the eigenvalues of $A$.

d) Compare your approximation to $\lambda_1$ and the value of $\lambda_1$ from part (c).

In [45]:
from matutil import listlist2mat

A = listlist2mat([[1,2,5,7],[2,9,3,7],[1,0,2,2],[7,3,9,1]])
eig_1_est = power_method(A, 100)

In [46]:
print(eig_1_est)


     0     1     2     3
------------------------
 0.394 0.792 0.105 0.454


In [55]:
lambda_1 = (A * eig_1_est)[0] / eig_1_est[0]

In [56]:
print(lambda_1 * eig_1_est)


    0    1    2    3
--------------------
 5.68 11.4 1.51 6.54


In [57]:
print(lambda_1 * eig_1_est)


    0    1    2    3
--------------------
 5.68 11.4 1.51 6.54


In [58]:
print(lambda_1)

14.402834217884141


In [41]:
import numpy as np

eigs = np.linalg.eig(np.array([[1,2,5,7],[2,9,3,7],[1,0,2,2],[7,3,9,1]]))

In [42]:
eigs

(array([ -6.52407415,  14.40283422,  -0.42946982,   5.55070975]),
 array([[-0.57465082,  0.39436403,  0.74337056,  0.26767904],
        [-0.25148857,  0.79218134, -0.20720852, -0.89716761],
        [-0.11336817,  0.1049683 , -0.55766706,  0.2266157 ],
        [ 0.77050477,  0.45377021,  0.30573237,  0.26848376]]))

In [59]:
eigs[0][1], eigs[1][:,1] # basically an exact match!

(14.402834217884141,
 array([ 0.39436403,  0.79218134,  0.1049683 ,  0.45377021]))

In [60]:
lambda_1 - eigs[0][1]

0.0

**Problem 12.14.6:** Prove:

**Lemma 12.14.7:** Suppose $A$ is an invertible matrix. The eigenvalues of $A^{-1}$ are the reciprocals of the eigenvalues of $A$.

Let $\lambda$ be any eigenvalue of $A$ and $\boldsymbol{v}$ be the corresponding eigenvector.

Then $A\boldsymbol{v} = \lambda\boldsymbol{v}$.

Multiply both sides of this equation on the left by $A^{-1}$:

$A^{-1}(A\boldsymbol{v}) = A^{-1}(\lambda\boldsymbol{v})$

Simplifying:

$\boldsymbol{v} = \lambda A^{-1}\boldsymbol{v}$

Multiplying both sides by $\lambda^{-1}$,

$\lambda^{-1}\boldsymbol{v} = A^{-1}\boldsymbol{v}$

$QED$

**Problem 12.14.8:** The lemma in Problem 12.14.6 shows that the eigenvalue of $A$ having smallest absolute value is the reciprocal of the eigenvalue of $A^{-1}$ having largest absolute value.

How can you use the power method to obtain an estimate of the eigenvalue of $A$ having smallest absolute value? You should not compute the inverse of $A$. Instead, use another approach: solving a matrix equation.

Use this approach on the matrix $A$ below:

$A = \begin{bmatrix}1 &2 &1 &9\\1 &3 &1 &3\\1 &2 &9 &5\\6 &4 &3 &1\end{bmatrix}$

We can use the power method to find the largest eigenvector $\boldsymbol{v}_1$ of $A$, and then rather than multiply $A^{-1}\boldsymbol{v}_1$ (which would mathematically give us what we want), we solve for the equation $A\boldsymbol{x} = \boldsymbol{v}_1$.  Then $\boldsymbol{x}$ is the smallest eigenvector of $A$ (the largest eigenvector for $A^{-1}$.

In [74]:
def inverse_power(A, mu):
    I = np.eye(A.shape[0])
    v = np.random.randn(A.shape[0])
    v = v / np.linalg.norm(v)
    for i in range(100):
        M = A - mu * I
        w = np.linalg.solve(M, v)
        v = w / np.linalg.norm(w)
        mu = v.dot(A.dot(v))
        if (A.dot(v) == mu * v).all():
            break
    return mu, v

In [75]:
A = np.array([[1,2,1,9],[1,3,1,3],[1,2,9,5],[6,4,3,1]])
mu, v = inverse_power(A, 1.0)

In [76]:
mu, v

(1.5234715699816992,
 array([ -5.54935979e-01,   8.04661145e-01,   1.54132036e-04,
         -2.11107737e-01]))

In [80]:
eigenvalues, eigenvectors = np.linalg.eig(A)
eigenvalues[-1], eigenvectors[:, -1] # exact match!

(1.5234715699817003,
 array([  5.54935979e-01,  -8.04661145e-01,  -1.54132036e-04,
          2.11107737e-01]))

**Problem 12.14.9:** Let $k$ be a number, $A$ an $n \times n$ matrix and $I$ the identity matrix. Let $\lambda_1, ..., \lambda_m (m \leq n)$ be the eigenvalues of $A$. What are the eigenvalues of $A - kI$? Justify your answer.

The eigenvalutes of $A - kI$ are the eigenvalues of $A$ shifted down by $k$:

$\begin{align}
(A - kI)\boldsymbol{v}_i &=A\boldsymbol{v}_i - kI\boldsymbol{v}\\&=\lambda_i\boldsymbol{v}_i - k\boldsymbol{v}_i\\&=(\lambda_i - k)\boldsymbol{v}_i\end{align}$

**Problem 12.14.10:** How can you use the lemma in Problem 12.14.8 and the result from 12.14.9 to address the following computational problem?

* _input:_ a matrix $A$ and a value $k$ that is an estimate of an eigenvalue $\lambda_i$ of $A$ (and is closer to $\lambda_i$ than to any other eigenvalue of $A$)
* _output:_ an even better estimate of that eigenvalue.

Show how to use this method on the following data:

$A = \begin{bmatrix}3 &0 &1\\4 &8 &1\\9 &0 &0\end{bmatrix}$, $k = 4$.

Actually the above method _does_ already do that (since it is frankly taken directly from the slides to solve exactly this problem and adapted for numpy:

In [81]:
A = np.array([[3,0,1],[4,8,1],[9,0,0]])
inverse_power(A, 4)

(4.8541019662496847, array([ 0.35577221, -0.66204524,  0.65963796]))

In [83]:
eigenvalues, eigenvectors = np.linalg.eig(A)
eigenvalues[1], eigenvectors[:, 1] # corresopnds to the second eigenvector/eigenvalue (down to insane precision since we're doing 100 iterations.)

(4.8541019662496847, array([ 0.35577221, -0.66204524,  0.65963796]))

**Problem 12.14.11:** (See weather Markov chain in book)

a) Give the transition matrix $A$ with row and column labels `{'S', 'R', 'F', 'W'}` (for "Sunny", "Rainy", "Foggy", and "Windy"). Entry $(i, j)$ of $A$ should be the probability of transitioning from state $j$ to state $i$. Construct a `Mat` in Python representing this matrix.

b) A probability distribution can be represented by a `{'S', 'R', 'F', 'W'}`-vector. If $\boldsymbol{v}$ is the probability distribution of the weather for today then $A\boldsymbol{v}$ is the probability distribution of the weather for tomorrow.

Write down the vector $\boldsymbol{v}$ representing the probability distribution in which it is Windy with probability 1. Calculate the vector $A\boldsymbol{v}$, which gives the probability distribution for the following day. Does it make sense in view of the diagram?

c) Write down the vector $\boldsymbol{v}$ representing the uniform distribution. calculate $A\boldsymbol{v}$.

d) What is the probability distribution over the weather states in 400 days given that the start probability distribution is uniform over the states?

e) Based on previous parts, name one eigenvalue and a corresponding eigenvector.


In [84]:
from mat import Mat
D = {'S', 'R', 'F', 'W'}
A = Mat((D, D), {
    ('S','S'): 0.5, ('R','S'): 0.2, ('W', 'S'): 0.3,
    ('S','R'): 0.2, ('R','R'): 0.6, ('F', 'R'): 0.2,
    ('F','F'): 0.4, ('W','F'): 0.6,
    ('S','W'): 0.1, ('F','W'): 0.8, ('W', 'W'): 0.1,
})

In [85]:
from vec import Vec

v = Vec(D, {'W': 1})

v_2 = A * v # distribution of tomorrow given windy today
print(v_2)


   F R   S   W
--------------
 0.8 0 0.1 0.1


These are exactly the transition probabilities from the 'Windy' node in the diagram.

In [86]:
uniform_dist = Vec(D, {k: 1 / len(D) for k in D})
print(uniform_dist)


    F    R    S    W
--------------------
 0.25 0.25 0.25 0.25


In [87]:
print(A * uniform_dist)


    F   R   S    W
------------------
 0.35 0.2 0.2 0.25


In [88]:
# inlining `power_method` to use an explicit starting vector.
x = uniform_dist
for i in range(400):
    x = A * x
    norm = (x * x) ** 0.5
    x = x / norm

print(x)


     F      R     S    W
------------------------
 0.798 0.0725 0.145 0.58


Thus, one eigenvector (the largest eigenvector) is

|     F|      R|     S|    W
|------------------------
| 0.798| 0.0725| 0.145| 0.58

A corresponding eigenvalue is...

In [90]:
(A * x)['R'] / x['R']

1.0

1.0 (which we already knew!)